# La syntaxe if let

---

## Un exemple trivial

- Dans le chapitre 2, nous avons croisé les conditions en Rust avec les mots-clefs `if` et `else`. Dans cette partie, nous allons decouvrir la syntaxe `if let` qui permet de tester si une variable correspond a un motif donné plutôt que de tester une condition. Considérons cet exemple simple :

In [5]:
let u: usize = 42;                                                                                                                                                       
if let 42 = u {                                                                                                                                                           
    println!("C'est le nombre 42");                                                                                                                                                   
} else if let 21 = u {
    println!("C'est le nombre 21");
} else {
    println!("C'est quelque chose d'autre !");
} 

C'est le nombre 42


()

> Ici, le motif de u est 42, on tombe dans le premier cas.

- Mais quel est l'intérêt d'utiliser `if let` alors qu'une simple condition `if` aurait suffit ici ?

In [4]:
let u: usize = 42;                                                                                                                                                        
if 42 == u {                                                                                                                                                           
    println!("C'est le nombre 42");                                                                                                                                                   
} else if 21 == u {
    println!("C'est le nombre 21");
} else {
    println!("C'est quelque chose d'autre !");
} 

C'est le nombre 42


()

> Dans ce cas, aucun ! Et on a même une limitation que l'on n'a pas avec un simple if, car avec if let, on ne peut pas par exemple écrire **if let 21 > u**, if let ne détermine que si 2 motifs sont identiques, il n'y a pas d'autres comparaisons tels supérieur a ou inférieur a possibles.

## L'inutilité des traits PartialEq et de Eq pour if let

### Test du motif d'une structure

- Seulement, avec un simple if, il est impératif que notre type implémente le trait PartialEq, et bien qu'il soit implémenté pour tous les types primitifs, il ne le sera pas forcément pour nos structures ou nos énumérations. Ainsi, ce code ne peut pas compiler :

In [6]:
struct MyStruct {
    u: usize,
}
let u: MyStruct = MyStruct {
    u: 42
};
let v: MyStruct = MyStruct {
    u: 42
};
if u == v {                                                                                                                                                           
    println!("C'est le nombre 42");                                                                                                                                                   
} else {
    println!("C'est quelque chose d'autre !");
} 

Error: an implementation of `PartialEq<_>` might be missing for `MyStruct`

- L'utilisation de if let a la place fonctionnerait bien ici :

In [7]:
// Pas de #[derive(PartialEq, Eq)]
struct MyStruct {
    u: usize,
}
let u: MyStruct = MyStruct {
    u: 42
};
// Les deux motifs sont identiques ici
if let MyStruct { u: 42 } = u {                                                                                                                                                           
    println!("C'est le nombre 42");                                                                                                                                                   
} else {
    println!("C'est quelque chose d'autre !");
} 

C'est le nombre 42


()

### Test d'un variant

- Tester si un variant spécifique d'une énumération se fait de la même manière 

In [19]:
// Pas de #[derive(PartialEq, Eq)]
enum Button {
    Up,
    Down,
    Middle,
}
let button = Button::Up;
if let Button::Up = button {
    println!("Button is UP");
}

Button is UP


()

### L'operateur OR et plusieurs motifs

- Il aussi possible de tester en fonction d'un set de motif en utilisant l'operateur OR **|** :

In [2]:
let u: usize = 12;
// Test de plusieurs motifs.
if let 3 | 12 = u {
    dbg!(u);
} else if let 4 | 13 | 1 = u {
    dbg!(u);
} else {
    println!("Unknown !");
}

struct S {
    i: usize,
    f: usize,
}
let s = S {i: 4, f: 6};
// Deux motifs sont possibles ici,
if let S{i: 4, f: 3} | S{i: 4, f: 6} = s {
    println!("One of these")
}

One of these


[src/lib.rs:109] u = 12


()

### Test d'une range

- Enfn on peut tester si un type primitif appartient a un certain interval :

In [21]:
    let u: usize = 12;
    // Inclusive ranges
    if let 1..=5 = u {
        println!("Between 1 to 5");
    } else if let 6..=12 = u {
        println!("Between 6 to 12");  
    }

Between 6 to 12


()

## Les motifs incomplets.

- Imaginons une structure qui contient 2 champs et admettons que je souhaite tester si son premier champ et le troisième correspond à des valeurs données et récupérer la valeur de son second champs, je pourrais tout à fait écrire ce code :

In [9]:
struct Coord {
    x: usize,
    y: usize,
    z: usize,
}
let p1 = Coord { x: 2, y: 6, z: 3 };
// Tests successifs de x et de z
if p1.x == 2 && p1.z == 3 {
    println!("la valeur de y vaut {}", p1.y);
}

la valeur de y vaut 6


()

- Cependant, if let me permet de tester facilement les champs x et z et de récupérer la valeur du champ y dans une autre variable.

In [10]:
struct Coord {
    x: usize,
    y: usize,
    z: usize,
}
let p1 = Coord { x: 2, y: 6, z: 3 };
// Le motif donne correspond il a p1 ?
if let Coord { x: 2, y: value, z: 3} = p1 {
    println!("la valeur de y vaut {}", value);
}

la valeur de y vaut 6


()

- Et dans le cas d'une énumération :

In [11]:
struct Voltage(usize);

// Pas de #[derive(PartialEq, Eq)]
enum PowerState {
    Up(Voltage),
    Down,
}

let c = PowerState::Up(Voltage(230));
// Le motif donne correspond a l'existance du variant Up, on recupere la valeur du voltage
if let PowerState::Up(Voltage(voltage_value)) = c {
    println!("the voltage value is {}", voltage_value);
} else {
    println!("the system is down");
}

the voltage value is 230


()

## Exercices

### Écrire une séquence de code avec if let qui permet de récupérer la valeur associée au variant Some :

In [12]:
enum MyOption {
    Some(usize),
    None,
}
let opt = MyOption::Some(11);

In [ ]:
// Ecrire le code ici

```
if let MyOption::Some(v) = opt {
    dbg!(v);
}
```

### Écrire une séquence de code avec if let qui permet de récupérer la valeur associée au variant Err :

In [13]:
enum MyResult {
    Ok(usize),
    Err(&'static str),
}
let res = MyResult::Err("Unexpected error... Woot ?!?");

In [ ]:
// Ecrire le code ici

```
if let MyResult::Err(s) = res {
    dbg!(s);
}
```